# Execução de modelos (LLMs) no Google Colab via Hugging Face
Este notebook carrega os modelos **CodeLLaMA-7B** e **deepseek-coder-6.7b** via Hugging Face, com quantização em 8-bit para rodar na GPU gratuita do Colab. Após o carregamento, uma API é exposta para uso do modelo no ChainForge.

In [ ]:
# --------------------------------------------------
# Instalar/atualizar dependências necessárias
# --------------------------------------------------

# P. compatibilidade e estabilidade
!pip install torch transformers accelerate bitsandbytes safetensors

# P. versões mais recentes
#!pip install -U bitsandbytes accelerate transformers einops safetensors


In [ ]:
# --------------------------------------------------
# Modelos
# --------------------------------------------------

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Usados no projeto:
# - "codellama/CodeLLaMA-7b-Instruct-hf"
# - "deepseek-ai/deepseek-coder-6.7b-instruct"

MODEL_CHOICE = ""

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHOICE)

# Configuração para rodar em 4-bit (mais leve para Colab T4)
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Carregar modelo já na GPU (se disponível)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_CHOICE,
    device_map="auto",
    quantization_config=bnb_config
)

# Criar pipeline direto
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",

    # Parâmetros importantes -------------------------------------------------------------------------------
    temperature=0.0,                      # Garante saída determinística (sem aleatoriedade)
    top_p=1.0,                            # Considera 100% do vocabulário possível (sem corte)
    do_sample=False,                      # Desativa amostragem aleatória - usa sempre o token mais provável
    repetition_penalty=1.0                # Mantém repetições normais (ideal para código estruturado)
    # ------------------------------------------------------------------------------------------------------

    #max_new_tokens=50,                   # limita a quantidade
    #eos_token_id=tokenizer.eos_token_id  # para encerrar a saída
)

print("Modelo carregado e pronto para uso:", MODEL_CHOICE)


## Exposição como API para uso no ChainForge

In [ ]:
# --------------------------------------------------
# Interface Gradio com link público
# --------------------------------------------------

import gradio as gr

def generate_code(prompt: str):
    try:
        out = generator(prompt, max_length=512, do_sample=True, temperature=0.0)
        if isinstance(out, list) and "generated_text" in out[0]:
            return out[0]["generated_text"]
        elif isinstance(out, list):
            return str(out[0])
        return str(out)
    except Exception as e:
        return f"Erro na geração: {e}"

iface = gr.Interface(
    fn=generate_code,
    inputs=gr.Textbox(lines=8, placeholder="Digite o prompt para gerar código..."),
    outputs=gr.Textbox(label="Código gerado"),
    title="LLMs no Colab via Hugging Face",
    description="Interface simples para testar prompts de geração de código"
)

# Criar link público temporário
iface.launch(share=True)